<a href="https://colab.research.google.com/github/mohamedfadul/DeepSearch-Finding-Truncated-Differential-Trails-Using-Genetic-Algorithms-and-Deep-Learning/blob/main/ANN_Baseline_model_using_data_up_to_round_8_to_predict_Perm5_round_15_Trail_Validity_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [2]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.12.0


In [4]:
#import tensorflow_docs as tfdocs
#import tensorflow_docs.plots
#import tensorflow_docs.modeling

In [5]:
!wget -O all_dataset_path.csv "https://drive.google.com/uc?export=download&id=14B7nCv7VKI-ZqmFzNl1_C-SqSI2RvOHM"

--2023-07-17 06:55:39--  https://drive.google.com/uc?export=download&id=14B7nCv7VKI-ZqmFzNl1_C-SqSI2RvOHM
Resolving drive.google.com (drive.google.com)... 142.251.16.139, 142.251.16.100, 142.251.16.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.16.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/43poekriglqae5leicv66r5pe8uo8p62/1689576900000/14347683118445347781/*/14B7nCv7VKI-ZqmFzNl1_C-SqSI2RvOHM?e=download&uuid=e15956d2-8d48-4427-bcad-d13ddad60fd5 [following]
--2023-07-17 06:55:40--  https://doc-0s-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/43poekriglqae5leicv66r5pe8uo8p62/1689576900000/14347683118445347781/*/14B7nCv7VKI-ZqmFzNl1_C-SqSI2RvOHM?e=download&uuid=e15956d2-8d48-4427-bcad-d13ddad60fd5
Resolving doc-0s-54-docs.googleusercontent.com (doc-0s-54-docs.googleusercontent.com)... 142.251.167.132, 2607

In [6]:
test_dataset =pd.read_csv ('all_dataset_path.csv')
test_dataset

,input16,input15,input14,input13,input12,input11,input10,input9,input8,input7,...,perm7,perm8,perm9,perm10,perm11,perm12,perm13,perm14,perm15,perm16
0,0,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1,1,0,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
2,1,1,1,1,1,0,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
3,1,0,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
4,1,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,0,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1084,0,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1085,1,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1086,1,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333


In [7]:
test_labels = test_dataset.pop('validity')

In [8]:
test_dataset['sboxes'] = ((test_dataset['sboxes']/(test_dataset['round']*8)))
test_dataset['sboxes']

0       0.741667
1       0.383333
2       0.966667
3       1.400000
4       2.416667
          ...   
1083    0.091667
1084    0.808333
1085    2.050000
1086    1.266667
1087    1.075000
Name: sboxes, Length: 1088, dtype: float64

In [9]:
test_dataset['round'] = (test_dataset['round'])/100

In [10]:
test_dataset

,input16,input15,input14,input13,input12,input11,input10,input9,input8,input7,...,perm7,perm8,perm9,perm10,perm11,perm12,perm13,perm14,perm15,perm16
0,0,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1,1,0,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
2,1,1,1,1,1,0,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
3,1,0,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
4,1,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,0,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1084,0,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1085,1,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333
1086,1,1,1,1,1,1,1,1,1,1,...,1,0.533333,0.2,0.8,0.066667,0.666667,0.466667,0,0.866667,0.933333


In [11]:
from keras.layers import Dropout
from keras import regularizers
from keras.regularizers import l2, l1
reg_param=0.001

def create_model():
  model = keras.Sequential([
        layers.Dense(512, activation='relu' , input_shape=[50]),
        #layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        #layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        #layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        #layers.Dropout(0.2),
      layers.Dense(1,activation='sigmoid' )
    ])
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)
  loss = tf.losses.mape
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.optimizers.Adam(learning_rate=0.001), metrics=(['accuracy'], ['Precision'],['Recall']))
  return model

In [12]:
validity_model = create_model()

In [13]:
validity_file_id = '1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW'  # Replace 'your_file_id' with the actual file ID

!wget -O validity_model_path.h5 "https://drive.google.com/uc?export=download&id={validity_file_id}"

--2023-07-17 06:55:40--  https://drive.google.com/uc?export=download&id=1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW
Resolving drive.google.com (drive.google.com)... 142.251.16.139, 142.251.16.100, 142.251.16.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.16.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/d2fjivl22l863r0u289e2btk6qboptbt/1689576900000/14347683118445347781/*/1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW?e=download&uuid=69eb6bb6-6cc0-4b81-9209-0648cdd8d360 [following]
--2023-07-17 06:55:41--  https://doc-04-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/d2fjivl22l863r0u289e2btk6qboptbt/1689576900000/14347683118445347781/*/1e2T7UBYXFRL31NYZCr7oM4TxP7HdCTmW?e=download&uuid=69eb6bb6-6cc0-4b81-9209-0648cdd8d360
Resolving doc-04-54-docs.googleusercontent.com (doc-04-54-docs.googleusercontent.com)... 142.251.167.132, 2607

In [14]:
validity_model.load_weights('validity_model_path.h5')

In [15]:
predictions = validity_model.predict(test_dataset)

34/34 [==============================] - 0s 2ms/step


In [16]:
validity_model.evaluate(test_dataset,test_labels)

34/34 [==============================] - 0s 4ms/step - loss: 0.1129 - accuracy: 0.9550 - precision: 0.9433 - recall: 0.7644


[0.11293242871761322,
 0.954963207244873,
 0.9432623982429504,
 0.7643678188323975]

In [17]:
from sklearn.metrics import confusion_matrix , classification_report
y_preds = validity_model.predict(test_dataset)
y_preds = np.round(y_preds)

print("Classification Report: \n", classification_report(test_labels, y_preds))

34/34 [==============================] - 0s 2ms/step
Classification Report: 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97       914
           1       0.94      0.76      0.84       174

    accuracy                           0.95      1088
   macro avg       0.95      0.88      0.91      1088
weighted avg       0.95      0.95      0.95      1088



In [18]:
predictions

array([[9.4574410e-01],
       [2.2349195e-03],
       [8.7366807e-01],
       ...,
       [3.0115249e-13],
       [2.7686068e-05],
       [2.4704735e-01]], dtype=float32)

In [19]:
import numpy as np
rounded = [int(np.round(x[0])) for x in predictions]
#rounded= np.array(rounded)
print(rounded)

[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [20]:
from sklearn.metrics import f1_score
f1_score(test_labels, rounded)

0.8444444444444444